In [1]:
from mlflow.tracking import MlflowClient

## Interact with mlflow client

In [2]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

2026/02/14 20:26:51 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/02/14 20:26:51 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


In [4]:
client.search_experiments()

[<Experiment: artifact_location='/Users/tianyuwu/Desktop/MyCompany/MLOps_research/mlops-project-0/02-experiment-tracking/mlruns/1', creation_time=1770757685948, experiment_id='1', last_update_time=1770757685948, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1770757273471, experiment_id='0', last_update_time=1770757273471, lifecycle_stage='active', name='Default', tags={}>]

In [5]:
from mlflow.entities import ViewType
runs = client.search_runs(
    experiment_ids="1",
    filter_string="",
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [6]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse:{run.data.metrics['rmse']:.4f}")

run id: 5dccc3dd131741f4879e64c84814d8eb, rmse:6.3020
run id: e37c86ba250649188f798fbd7b1df660, rmse:6.3020
run id: c4ad768b6ee84325b73e5bd8aae4193e, rmse:6.3082
run id: b0e604e798504e9ca1a70eadf6ed30eb, rmse:6.3096
run id: 27514bfa47ce4877971948267a6832fb, rmse:6.3111


## Interact with model registry

In [7]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [12]:
run_id = "729eb962226b4a0dbd67ab47f2fae18f"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2026/02/14 20:42:44 WARNING mlflow.tracking._model_registry.fluent: Run with id 729eb962226b4a0dbd67ab47f2fae18f has no artifacts at artifact path 'model', registering model based on models:/m-1f647287b083457e9a11aa08ec1b86a5 instead
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1771130564946, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1771130564946, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='729eb962226b4a0dbd67ab47f2fae18f', run_link=None, source='models:/m-1f647287b083457e9a11aa08ec1b86a5', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [13]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: Staging
version: 3, stage: None


/var/folders/ly/3srjzss91pl5_7m5lmbc9km00000gn/T/ipykernel_83779/669935608.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [14]:
model_version = 3
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/var/folders/ly/3srjzss91pl5_7m5lmbc9km00000gn/T/ipykernel_83779/1957332551.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1771130564946, current_stage='Staging', deployment_job_state=None, description=None, last_updated_timestamp=1771130607579, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='729eb962226b4a0dbd67ab47f2fae18f', run_link=None, source='models:/m-1f647287b083457e9a11aa08ec1b86a5', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [15]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1771130564946, current_stage='Staging', deployment_job_state=None, description='The model version 3 was transitioned to Staging on 2026-02-14', last_updated_timestamp=1771130656778, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='729eb962226b4a0dbd67ab47f2fae18f', run_link=None, source='models:/m-1f647287b083457e9a11aa08ec1b86a5', status='READY', status_message=None, tags={}, user_id=None, version=3>

## Comparing versions and selecting the new "Production" model

### Test models on unseen dataset

In [19]:
from sklearn.metrics import root_mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

In [20]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [21]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/Users/tianyuwu/Desktop/MyCompany/MLOps_research/mlops-project-0/02-experiment-tracking/preprocessor'

In [22]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [23]:
X_test = preprocess(df, dv)

In [24]:
target = "duration"
y_test = df[target].values

In [25]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

CPU times: user 501 ms, sys: 281 ms, total: 782 ms
Wall time: 1.27 s


{'rmse': 6.659623830022514}

In [26]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

CPU times: user 6.32 s, sys: 448 ms, total: 6.77 s
Wall time: 7.2 s


{'rmse': 6.894653105581671}

In [27]:
client.transition_model_version_stage(
    name=model_name,
    version=3,
    stage="Production",
    archive_existing_versions=True
)

/var/folders/ly/3srjzss91pl5_7m5lmbc9km00000gn/T/ipykernel_83779/684772386.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1771130564946, current_stage='Production', deployment_job_state=None, description='The model version 3 was transitioned to Staging on 2026-02-14', last_updated_timestamp=1771131075460, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='729eb962226b4a0dbd67ab47f2fae18f', run_link=None, source='models:/m-1f647287b083457e9a11aa08ec1b86a5', status='READY', status_message=None, tags={}, user_id=None, version=3>